<a href="https://colab.research.google.com/github/adrian-lison/gnn-community-detection/blob/master/Notebooks/LGNN_Semi_supervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model #4 LGNN
----------


In [21]:
#!pip install dgl

## Imports

In [22]:
# dgl
import dgl
import dgl.function as fn
from dgl import DGLGraph
from dgl.data import citation_graph as citegrh

# pytorch
import torch as th
import torch.nn as nn
import torch.nn.functional as F

# other
import time
import numpy as np
import random as rng
import scipy.sparse as ss
import networkx as nx
import pickle

from sklearn.metrics import accuracy_score as acc

## GNN Definition

In [23]:
# Return a list containing features gathered from multiple radius.
import dgl.function as fn
def aggregate_radius(radius, g, z):
    # initializing list to collect message passing result
    z_list = []
    g.ndata['z'] = z
    # pulling message from 1-hop neighbourhood
    g.update_all(fn.copy_src(src='z', out='m'), fn.sum(msg='m', out='z'))
    z_list.append(g.ndata['z'])
    for i in range(radius - 1):
        for j in range(2 ** i):
            #pulling message from 2^j neighborhood
            g.update_all(fn.copy_src(src='z', out='m'), fn.sum(msg='m', out='z'))
        z_list.append(g.ndata['z'])
    return z_list

In [24]:
class LGNNCore(nn.Module):
    def __init__(self, in_feats, out_feats, radius):
        super(LGNNCore, self).__init__()
        self.out_feats = out_feats
        self.radius = radius

        self.linear_prev = nn.Linear(in_feats, out_feats)
        self.linear_deg = nn.Linear(in_feats, out_feats)
        self.linear_radius = nn.ModuleList(
                [nn.Linear(in_feats, out_feats) for i in range(radius)])
        self.linear_fuse = nn.Linear(in_feats, out_feats)
        self.bn = nn.BatchNorm1d(out_feats)

    def forward(self, g, feat_a, feat_b, deg, pm_pd):
        # term "prev"
        prev_proj = self.linear_prev(feat_a)
        # term "deg"
        deg_proj = self.linear_deg(deg * feat_a)

        # term "radius"
        # aggregate 2^j-hop features
        hop2j_list = aggregate_radius(self.radius, g, feat_a)
        # apply linear transformation
        hop2j_list = [linear(x) for linear, x in zip(self.linear_radius, hop2j_list)]
        radius_proj = sum(hop2j_list)

        # term "fuse"
        fuse = self.linear_fuse(th.mm(pm_pd, feat_b))

        # sum them together
        result = prev_proj + deg_proj + radius_proj + fuse

        # skip connection and batch norm
        n = self.out_feats
        result = th.cat([result[:, :n], F.relu(result[:, n:])], 1)
        result = self.bn(result)

        return result

In [25]:
class LGNNLayer(nn.Module):
    def __init__(self, in_feats, in_feats_lg, out_feats, radius):
        super(LGNNLayer, self).__init__()
        self.g_layer = LGNNCore(in_feats, out_feats, radius)
        self.lg_layer = LGNNCore(in_feats_lg, out_feats, radius)

    def forward(self, g, lg, x, lg_x, deg_g, deg_lg, pm_pd):
        next_x = self.g_layer(g, x, lg_x, deg_g, pm_pd)
        pm_pd_y = th.transpose(pm_pd, 0, 1)
        next_lg_x = self.lg_layer(lg, lg_x, x, deg_lg, pm_pd_y)
        return next_x, next_lg_x

In [125]:
class LGNN(nn.Module):
    def __init__(self, radius, g_feat, lg_feat, dropout = 0.2):
        super(LGNN, self).__init__()
        self.layer1 = LGNNLayer(g_feat.shape[1], lg_feat.shape[1], 21, radius)  # input
        self.layer2 = LGNNLayer(21, 21, 21, radius)  # hidden size is 16
        self.layer3 = LGNNLayer(21, 21, 21, radius)
        self.linear = nn.Linear(21, 21, 7)  # predict seven classes
        self.dropout = dropout
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

        # compute the degrees
        self.deg_g = g.in_degrees().float().unsqueeze(1)
        self.deg_lg = lg.in_degrees().float().unsqueeze(1)  

    def forward(self, g, lg, pm_pd, g_feat, lg_feat):
        x, lg_x = g_feat, lg_feat
        x, lg_x = self.layer1(g, lg, x, lg_x, self.deg_g, self.deg_lg, pm_pd)
        if self.dropout>0: x = self.dropout1(x)
        x, lg_x = self.layer2(g, lg, x, lg_x, self.deg_g, self.deg_lg, pm_pd)
        if self.dropout>0: x = self.dropout2(x)
        x, lg_x = self.layer3(g, lg, x, lg_x, self.deg_g, self.deg_lg, pm_pd)
        return self.linear(x)

## Data Loading

In [115]:
#Loading CORA
data = citegrh.load_cora()
features = th.FloatTensor(data.features)
labels = th.LongTensor(data.labels)
mask = th.ByteTensor(data.train_mask)
g = data.graph
lg_feat = np.vstack([features[e[0],:]*features[e[1],:] for e in data.graph.edges])

In [117]:
g = DGLGraph(g) # turn networkx graph into DGL graph
lg = g.line_graph(backtracking=False)

In [118]:
matrix = ss.lil_matrix((data.graph.number_of_nodes(),data.graph.number_of_edges()))
for s,d in zip(g.edges()[0],g.edges()[1]):
    matrix[s,g.edge_id(s,d)] = -1
    matrix[d,g.edge_id(s,d)] = 1
inputs_pmpd = ss.coo_matrix(matrix,dtype="int64")
inputs_pmpd

<2708x10556 sparse matrix of type '<class 'numpy.int64'>'
	with 21112 stored elements in COOrdinate format>

In [119]:
np.unique(labels)

array([0, 1, 2, 3, 4, 5, 6], dtype=int64)

In [120]:
print('We have %d nodes.' % g.number_of_nodes())
print('We have %d edges.' % g.number_of_edges())

We have 2708 nodes.
We have 10556 edges.


## Select Training Set

In [121]:
percentage_train = 0.02

with open("data/cora_permutation1.pickle","rb") as f:
    perm1 = pickle.load(f)
mask = np.zeros(g.number_of_nodes())
mask[perm1[range(int(percentage_train*g.number_of_nodes()))]] = 1
mask = th.ByteTensor(mask)

## Training

In [1]:
# create the model
degrees = True
if degrees:
    # use degree as the input feature
    # compute the degrees
    g_feat = g.in_degrees().float().unsqueeze(1)
    lg_feat = lg.in_degrees().float().unsqueeze(1)  
else:
    # use astracts as input feature
    g_feat = features
    lg_feat = th.FloatTensor(lg_feat)

net = LGNN(radius=3, g_feat = g_feat, lg_feat = lg_feat, dropout=0.0)
# define the optimizer
optimizer = th.optim.Adam(net.parameters(), lr=0.01, weight_decay=0.1)

# a util function to convert a scipy.coo_matrix to torch.SparseFloat
def sparse2th(mat):
    value = mat.data
    indices = th.LongTensor([mat.row, mat.col])
    tensor = th.sparse.FloatTensor(indices, th.from_numpy(value).float(), mat.shape)
    return tensor

all_logits = []
all_losses = []
all_losses_val = []

pmpd = sparse2th(inputs_pmpd)
lg = g.line_graph(backtracking=False)

# train
for epoch in range(1000):

    # Compute loss for test nodes (only for validation, not used by optimizer)
    net.eval()
    prediction = F.log_softmax(net(g, lg, pmpd,  g_feat = g_feat, lg_feat = lg_feat),1)
    val_loss = F.nll_loss(prediction.detach()[1-mask], labels[1-mask])
    all_losses_val.append(val_loss.item())
    val_acc = acc(labels[1-mask],prediction.detach()[1-mask].numpy().argmax(axis=1))
    net.train()

    logits = net(g, lg, pmpd,  g_feat = g_feat, lg_feat = lg_feat)
    # Save logits for visualization later
    all_logits.append(logits.detach())
    logp = F.log_softmax(logits, 1)

    # Compute loss for train nodes
    loss = F.nll_loss(logp[mask], labels[mask])
    all_losses.append(loss.detach().item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print('Epoch %d | Loss: %.4f | Total: %.4f | Acc: %.4f' % (epoch, loss.item(), val_loss.item(),val_acc))

NameError: name 'g' is not defined

In [223]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
traces= [go.Scatter(x=list(range(len(all_losses))),y=all_losses,name="train",mode="lines"),
go.Scatter(x=list(range(len(all_losses_val))),y=all_losses_val,name="val",mode="lines")]
layout = go.Layout(title="Loss Evolution", xaxis=dict(title="Step"), yaxis=dict(title="NLL-Loss",range=[0, 3]))
fig = dict(data=traces, layout=layout)
iplot(fig)

## Evaluation

In [218]:
net.eval() # Set net to evaluation mode (deactivates dropout)
final_prediction = F.log_softmax(net(g, lg, pmpd,  g_feat = g_feat, lg_feat = lg_feat),1).detach()

pred_sets = {"All ":final_prediction,"Train":final_prediction[mask],"Test":final_prediction[1-mask]}
label_sets = {"All ":labels,"Train":labels[mask],"Test":labels[1-mask]}
eval_functions = {"NLL-Loss":lambda y,x: F.nll_loss(x,y),"Accuracy":lambda y,x: acc(y,x.numpy().argmax(axis=1))}

for name,func in eval_functions.items():
    eval_message = f"\n{name}:\n"
    for subset in pred_sets.keys():
        eval_message += f" {subset}: {func(label_sets[subset],pred_sets[subset]):.4f} |"
    print(eval_message)


NLL-Loss:
 All : 6.1416 | Train: 0.0284 | Test: 6.2660 |

Accuracy:
 All : 0.3741 | Train: 1.0000 | Test: 0.3613 |
